In [1]:
from database.market import Market
from transformer.column_transformer import ColumnTransformer
from transformer.date_transformer import DateTransformer
from utils.date_utils import DateUtils
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime, timedelta, timezone
from tqdm import tqdm
import math
import numpy as np
import statistics

In [2]:
## Loading Constants
start = (datetime.now() - timedelta(days=730)).strftime("%Y-%m-%d")
end = datetime.now().strftime("%Y-%m-%d")
# Loading Databases
market = Market()
market.connect()
tickers = market.retrieve_data("sp500").sort_values("Symbol")
market.close()
model_range = range(len(tickers))
datasets = [
#                 "pdr",
            "tiingo"
            ,"finnhub"
           ]

In [3]:
refined_daily_classification = []
refined_quarterly_classification = []
refined_weekly_classification = []
refined_model_regression = []
market.connect()
for i in tqdm(model_range):
    try:
        refined_regression = []
        for dataset in datasets:
            ticker = tickers["Symbol"][i]
            if "." in ticker:
                ticker = ticker.replace(".","-")
            prices = market.retrieve_price_data("portfolio_{}_prices".format(dataset),ticker)
            if dataset == "pdr":
                prices = ColumnTransformer.rename_columns(prices, " ")
            else:
                prices = ColumnTransformer.rename_columns(prices, "_")
            prices = DateTransformer.convert_to_date(dataset,prices,"date")
            prices.reset_index(inplace=True)
            relev = prices[["date","adjclose"]]
            relev["ticker"] = ticker
            relev.sort_values("date",inplace=True)
            relev.rename(columns={"adjclose":dataset},inplace=True) 
            relev["date"] = [datetime.strptime(str(x).split(" ")[0],"%Y-%m-%d") for x in relev["date"]]
            ## daily transformations
            refined_regression.append(relev)
        base = refined_regression[0]
        for rr in refined_regression[1:]:
            base = base.merge(rr,on=["date","ticker"],how="left")
        adjclose = []
        for row in base.iterrows():
            values = []
            for x in datasets:
                try:
                    values.append(row[1][x])
                except:
                    continue
            adjclose.append(np.nanmean(values))
        base["adjclose"] = adjclose
        relev = base.copy()
        relev["week"] = [x.week for x in relev["date"]]
        relev["quarter"] = [x.quarter for x in relev["date"]]
        relev["year"] = [x.year for x in relev["date"]]
        refined_model_regression.append(relev.copy())
    except Exception as e:
        print(str(e),ticker)
market.close()

  0%|                                                                     | 0/505 [00:00<?, ?it/s]<ipython-input-3-1932727f8913>:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  relev["ticker"] = ticker
<ipython-input-3-1932727f8913>:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  relev.sort_values("date",inplace=True)
c:\users\eric\appdata\local\programs\python\python38\lib\site-packages\pandas\core\frame.py:4125: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.py

't' YUM
't' ZBRA
't' ZBH
't' ZION
't' ZTS


In [4]:
base = pd.concat(refined_model_regression)

In [5]:
base["date"].max()

Timestamp('2021-07-30 00:00:00')

In [6]:
daily_base = base[base["date"]>datetime(2021,7,28)]

In [7]:
daily_base["date"].min()

Timestamp('2021-07-29 00:00:00')

In [8]:
market.connect()
prices = market.retrieve_price_data("portfolio_{}_prices".format(dataset),"AAPL")
market.close()

In [8]:
daily_base

,date,tiingo,ticker,finnhub,adjclose,week,quarter,year
596,2021-07-29,198.17,MMM,197.94,198.055,30,3,2021
597,2021-07-30,197.94,MMM,NaN,197.940,30,3,2021
596,2021-07-29,121.09,ABT,120.98,121.035,30,3,2021
597,2021-07-30,120.98,ABT,NaN,120.980,30,3,2021
596,2021-07-29,118.87,ABBV,116.30,117.585,30,3,2021
...,...,...,...,...,...,...,...,...
597,2021-07-30,68.25,XEL,NaN,68.250,30,3,2021
596,2021-07-29,147.25,XLNX,149.84,148.545,30,3,2021
597,2021-07-30,149.84,XLNX,NaN,149.840,30,3,2021
596,2021-07-29,125.06,XYL,125.85,125.455,30,3,2021


In [9]:
market.connect()
market.store_data("portfolio_prices",daily_base.groupby(["date","ticker"]).mean().reset_index())
market.close()

In [10]:
final = base.pivot_table(index=["year","quarter","week"],values="adjclose",columns="ticker").reset_index()
final.fillna(-99999,inplace=True)
for timeframe in ["week"]:
    if timeframe == "week":
        relev = final.groupby(["year","quarter","week"]).mean().reset_index()
    else:
        relev = final.groupby(["year","quarter"]).mean().reset_index()
    relev.reset_index(drop=True,inplace=True)
    name = "portfolio_dataset_{}_regression".format(timeframe)

In [11]:
weekly_set = relev[(relev["year"]==2021) & (relev["week"] > 28)]

In [12]:
weekly_set

ticker,year,quarter,week,A,AAL,AAP,AAPL,ABBV,ABC,ABMD,...,WRK,WST,WU,WY,WYNN,XEL,XLNX,XOM,XRAY,XYL
118,2021,3,29,149.922222,20.845556,210.601111,146.252222,116.995556,118.257778,323.513333,...,48.305556,373.578983,23.026667,33.990,106.555556,67.747778,134.538889,56.787778,63.016667,121.741111
119,2021,3,30,151.810000,21.174000,213.229000,146.135000,117.745000,121.822000,324.756000,...,49.477000,394.732000,23.255000,34.395,100.383000,68.696000,141.854000,58.115000,65.247000,124.551000


In [13]:
name

'portfolio_dataset_week_regression'

In [14]:
market.connect()
market.store_data(name,weekly_set)
market.close()